In [65]:
import numpy as np
import math
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import scipy.linalg

In [66]:
def symmetric_generator(n):
    a = np.random.randn(n, n)
    return np.tril(a) + np.tril(a, -1).T

In [67]:
def vec_mod(n):
    return math.sqrt(sum([i ** 2 for i in n]))

In [68]:
def numpy_eig(a):
    w, v = np.linalg.eigh(a)
    return w, v

In [69]:
def power_eig(a, iterations, epsilon):
    x = np.random.rand(a.shape[0])
    x = x / np.linalg.norm(x)
    dx0 = 0
    dx1 = 1
    i = 0
    while (i < iterations and abs(dx0 - dx1) > epsilon):
        dx0 = vec_mod(x)
        
        y = a @ x
        y_norm = np.linalg.norm(y)
        x = y / y_norm
        l = np.transpose(x) @ a @ x
        
        dx1 = vec_mod(x)
        i += 1
    return l, x    

In [70]:
def inverse_eig(a, iterations, epsilon):
    x = np.random.rand(a.shape[0])
    x = x / np.linalg.norm(x)
    i = 0
    dx0 = 0
    dx1 = 1
    (lu, piv) = scipy.linalg.lu_factor(a)
    while (i < iterations and abs(dx0 - dx1) > epsilon):
        dx0 = vec_mod(x)
        w = scipy.linalg.lu_solve((lu, piv), x)
        # w = x @ np.linalg.inv(a) 
        x = w / np.linalg.norm(w)
        l = np.transpose(x) @ a @ x
        
        dx1 = vec_mod(x)
        i += 1
    return l, x    

#### Symetryczna macierz liczb rzeczywistych

In [76]:
a = symmetric_generator(3)
print(a)

[[-1.87618085  1.50466469 -0.76438031]
 [ 1.50466469 -0.72151822 -0.58298238]
 [-0.76438031 -0.58298238  0.73585262]]


#### Funkcja numpy.linalg.eigh

In [77]:
w, v = numpy_eig(a)

for i in range(len(w)):
    print(w[i], v[:,i])

-2.93676246401 [-0.83459521  0.54389123 -0.08736803]
-0.399598377581 [ 0.38516191  0.68954897  0.61332905]
1.4745143818 [ 0.39382883  0.47823065 -0.78498045]


#### Sprawdzenie

In [78]:
v @ np.diag(w) @ np.transpose(v) # Tw o symetrycznym zadaniu własnym

array([[-1.87618085,  1.50466469, -0.76438031],
       [ 1.50466469, -0.72151822, -0.58298238],
       [-0.76438031, -0.58298238,  0.73585262]])

#### Power Iteration

In [86]:
l0, u1 = power_eig(a, 1000, 0.00000001)
l1 = (a @ u1) / u1 # Ax = lx -> l = Ax / x
print (l0, np.mean(l1), u1)

-2.70965900258 -2.20009214622 [-0.83934024  0.42570245 -0.33806123]


#### Size vs Time

In [ ]:
size = [10, 50, 100, 150, 300, 400, 500, 750, 1000]
time = [0, 0, 0, 0, 0, 0, 0, 0, 0]
for j in range(5):
    for i in range(len(size)):
        a = symmetric_generator(size[i])    
        start = timer()
        power_eig(a, 100, 0.0000001)
        end = timer()
        time[i] += (end - start)
time = [i/5 for i in time]

In [ ]:
plt.plot(size, time)
plt.xlabel('Size')
plt.ylabel('Time')
plt.show()

#### Inverse Iteration

In [104]:
a = symmetric_generator(2)
print(a, '\n')

v = np.zeros((2,2))
l, u = inverse_eig(a, 1000, 0.00000001)
l1, u1 = power_eig(a, 1000, 0.00000001)
v[0][0] = u[0]
v[1][0] = u[1]
v[0][1] = u1[0]
v[1][1] = u1[1]

print(v @ np.diag([l, l1]) @ np.transpose(v))


[[-1.51037671 -1.17570634]
 [-1.17570634 -0.91081316]] 

[[-1.51273708 -1.17509416]
 [-1.17509416 -0.90846629]]


In [63]:
b = [2, 1, 3]
a = symmetric_generator(3)
print(a, '\n')

x = b @ np.linalg.inv(a)

p, l, u = scipy.linalg.lu(a)
y = p @ b @ np.linalg.inv(l)
x2 = y @ np.linalg.inv(u)

print(a @ x, '\n', a @ x2)

[[-1.34991505 -2.33535323  1.6566817 ]
 [-2.33535323  1.60776074 -1.23423061]
 [ 1.6566817  -1.23423061  0.34716913]] 

[ 2.  1.  3.] 
 [-8.49820018  9.35749092 -2.70584247]


#### Rayleigh Quotient

In [ ]:
def rayleigh_eig(a, epsilon)

In [ ]:
function x = rayleigh(A, epsilon, mu, x)
  x = x / norm(x);
  % the backslash operator in Octave solves a linear system
  y = (A - mu * eye(rows(A))) \ x; 
  lambda = y' * x;
  mu = mu + 1 / lambda
  err = norm(y - lambda * x) / norm(y)

  while err > epsilon
    x = y / norm(y);
    y = (A - mu * eye(rows(A))) \ x;
    lambda = y' * x;
    mu = mu + 1 / lambda
    err = norm(y - lambda * x) / norm(y)
  end

end